In [54]:
# pacakges 
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
from statsmodels.compat import lzip
import matplotlib.pyplot as plt 
import seaborn as sns 
import matplotlib.pyplot as plt
from statsmodels.graphics.regressionplots import plot_leverage_resid2


In [34]:
# reading final csv file 
df = pd.read_csv("walmart_variables.csv")
df = df.drop('Unnamed: 0' , axis = 1)

In [58]:
# 현재 있는 데이터로 OLS 돌려보기 
result = sm.OLS.from_formula("units ~ C(store_nbr) : C(item_nbr) + tmin + dewpoint + cool + preciptotal + resultspeed + resultdir + avgspeed + C(dayofweek) + C(month) + C(holiday) + 0" , data= df).fit()
summary = result.summary()
print(summary())

KeyboardInterrupt: 

In [51]:
def model_check(result) : 
    '''cpu 대용량 필요...'''
    '''residual normality test'''
    Chi_square , P_value = sms.omni_normtest(result.resid)
    normality = pd.DataFrame(columns = [["Residual Normality Test" , "Residual Normality Test"],["Chi Square" , "P-Value"]])
    normality.loc[len(normality)] = [round(Chi_square,2) , round(P_value,2) ]

    '''heteroskedacity test : 종속 변수 값의 분산이 독립 변수의 값과 관계 없이 어느 정도 고정인가?''' 
    a, b, c, d = sms.het_white(result.resid, result.model.exog)
    heteroskedacity = pd.DataFrame(columns =[["Heteroskedacity Test","Heteroskedacity Test","Heteroskedacity Test","Heteroskedacity Test"],['Lagrange Statistic' ,'P-Value', 'F-Value', 'F P-Value' ]])
    heteroskedacity.loc[len(heteroskedacity)] = [round(a ,2 ), round(b, 2), round(c, 2), round(d ,2)]
  
    '''multicollinearity test : condition number 구함''' 
    print("")
    multicollinearity = pd.DataFrame(columns = [["Multicollinearity" ],["condition number"]])
    multicollinearity.loc[len(multicollinearity)] = [round(np.linalg.cond(result.model.exog) ,2)]
    x = pd.concat([normality, heteroskedacity, multicollinearity], axis = 1)
    
    return x


In [56]:
def influence_factor(result) : 
    fig, ax = plt.subplots(figsize=(8,6))
    fig = plot_leverage_resid2(result, ax = ax)

In [105]:
df.head()

,store_nbr,item_nbr,units,station_nbr,tmin,dewpoint,cool,preciptotal,resultspeed,resultdir,avgspeed,dayofweek,month,holiday
0,1,1,0,1,31.0,36.0,0.0,0.05,3.6,20.0,4.6,weekend,1,holiday
1,1,2,0,1,31.0,36.0,0.0,0.05,3.6,20.0,4.6,weekend,1,holiday
2,1,3,0,1,31.0,36.0,0.0,0.05,3.6,20.0,4.6,weekend,1,holiday
3,1,4,0,1,31.0,36.0,0.0,0.05,3.6,20.0,4.6,weekend,1,holiday
4,1,5,0,1,31.0,36.0,0.0,0.05,3.6,20.0,4.6,weekend,1,holiday


In [102]:
# cross validation using k-fold method 
from sklearn.model_selection import KFold

dfX = df.drop('units' , axis = 1)
dfY = df['units']

cv = KFold(20)
for i , (train_index, test_index) in enumerate(cv.split(df)): 
    x_train, y_train = dfX[train_index] , dfY[test_index]
    x_test , y_test = dfX[test_index] , dfY[test_index]
    

KeyError: '[ 215296  215297  215298 ... 4305909 4305910 4305911] not in index'